

# Método Newton

O método de Newton obtém a solução aproxima do sistema de equações não lineares através do seguinte
processo iterativo:

**Para** $k ← 0$ **ate** $k_{max}$
 1. $
\mathbf{J}(\mathbf{x}^{(k)})\mathbf{s}= - \mathbf{F}(\mathbf{x}^{(k)}) \quad \quad (1)$
 2. $\mathbf{x}^{(k+1)} = \mathbf{x}^k + \mathbf{\Delta x}$
 3. **Se** $\|F(\mathbf{x}^{(k+1)})| < ϵ $ **E** $\|\Delta x\| < ϵ$
    - **Fim**


onde $\mathbf{J}$ é a matriz jacobiana dada por:

$$
\mathbf{J}(\mathbf{x})^{(k)}_{ij} = \frac{\partial f_i(\mathbf{x}^{(k)})}{\partial x_j}
$$

**Exemplo**:

Resolva o sistema linear abaixo:
\begin{align}
(1/b)e^{bx_0} - x_1 &= 0 \\
x_0^2 + x_1^2 - 1   &= 0
\end{align}
considerando $b=2$ e $\mathbf{x}^{0} = (1,1)$.

*Solução:*

In [ ]:
import numpy as np


def ComputeResiduo(x):
    F = np.zeros([2, 1])
    b = 2.0
    F[0] = (1.0 / b) * np.exp(b * x[0]) - x[1]
    F[1] = x[0] ** 2 + x[1] ** 2 - 1
    return F


def ComputeExJacobian(x):
    J = np.zeros([2, 2])
    b = 2.0
    J[0][0] = np.exp(b * x[0])
    J[0][1] = -1
    J[1][0] = 2 * x[0]
    J[1][1] = 2 * x[1]
    return J


def Newton(x0, kmax, tolerance):
    print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error"))
    for i in range(kmax):
        J = ComputeExJacobian(x0)
        F = ComputeResiduo(x0)
        dx = np.linalg.solve(J, -F)
        x = x0 + dx
        error = np.linalg.norm(x - x0) / np.linalg.norm(x)
        print("%-5d  %-8.8f %-8.8f %-8.8E" % (i, x[0], x[1], error))
        if error < tolerance:
            break
        x0 = x
    return x


x0 = np.ones([2, 1])
x = Newton(x0, 100, 1.0e-12)

Iter   x1         x2         error   
0      0.61920292 0.88079708 3.70604662E-01
1      0.39415744 0.94862308 2.28808918E-01
2      0.32519914 0.94815663 6.87964976E-02
3      0.31966506 0.94754696 5.56747364E-03
4      0.31963154 0.94754191 3.38954819E-05
5      0.31963154 0.94754191 1.24036516E-09
6      0.31963154 0.94754191 0.00000000E+00


Claramente a iteração de Newton requer o calculo da matriz Jacobiana, mas nós podemos fornecer apenas uma função
F(x). As entradas do Jacobiano são, no entanto, derivadas que podem ser aproximadas por diferenças finitas. Seja $\mathbf{u}_j\in \mathbb{R}^n$ um vetor com entrada 1 na posição $j$ e zero nas demais. Se $\delta$ uma valor pqueno diferente de zero, então uma entrada da matriz $\mathbf{J}$ pode ser aproximada por  

$$
\mathbf{J}(\mathbf{x})^{(k)}_{ij} \approx  \frac{F_i(\mathbf{x}^{(k)} + \delta \mathbf{u}_j)- F_i(\mathbf{x}^{(k)})}{\delta }
$$

Vejamos o que acontece com e exemplo anterior:


In [ ]:
def ComputeApproxJacobian(x):
    eps = 1.0e-4
    J = np.matrix([2, 2])
    u = np.zeros([2, 1])
    for i in range(2):
        for j in range(2):
            u[j] = eps
            J[i][j] = (ComputeResiduo(x + u) - ComputeResiduo(x)) / eps
            u[j] = 0.0
    return J


def NewtonApprox(x0, kmax, tolerance):
    print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error"))
    for i in range(kmax):
        J = ComputeApproxJacobian(x0)
        F = ComputeResiduo(x0)
        dx = np.linalg.solve(J, -F)
        x = x0 + dx
        error = np.linalg.norm(x - x0) / np.linalg.norm(x)
        print("%-5d  %-8.8f %-8.8f %-8.8E" % (i, x[0], x[1], error))
        if error < tolerance:
            break
        x0 = x
    return x


x0 = np.ones([2, 1])
x = Newton(x0, 100, 1.0e-12)

Iter   x1         x2         error   
0      0.61920292 0.88079708 3.70604662E-01
1      0.39415744 0.94862308 2.28808918E-01
2      0.32519914 0.94815663 6.87964976E-02
3      0.31966506 0.94754696 5.56747364E-03
4      0.31963154 0.94754191 3.38954819E-05
5      0.31963154 0.94754191 1.24036516E-09
6      0.31963154 0.94754191 0.00000000E+00


# Método de Broyden

A avaliação da matriz jacobiana em cada iterações não linear pode demandar alto custo computacional.
O método de Broyden aproxima o jacobiano com a seguinte fórmula:

$$
\mathbf{B}^{k+1} = \mathbf{B}^{k} + \frac{(\mathbf{y} - \mathbf{B}^k)\mathbf{s}^T }{\mathbf{s}^T\mathbf{s}} = \mathbf{B}^{k} + \frac{\mathbf{F}(\mathbf{x}^{k+1})\mathbf{s}^T }{\mathbf{s}^T\mathbf{s}} \quad k=1,2,3,...
$$

onde $\mathbf{y} = \mathbf{F}(\mathbf{x}^{k+1}) - \mathbf{F}(\mathbf{x}^{k})$ e $\mathbf{B}^0 = \mathbf{J}(\mathbf{x}^{0})$.

Abaixo, segue o algoritmos em python:

In [ ]:
def broyden(x0, kmax, tolerance):
    B = ComputeExJacobian(x0)
    F0 = ComputeResiduo(x0)
    for i in range(kmax):
        dx = np.linalg.solve(B, -F0)
        x = x0 + dx
        error = np.linalg.norm(x - x0) / np.linalg.norm(x)
        print("%-5d  %-8.8f %-8.8f %-8.8E" % (i, x[0], x[1], error))
        if error < tolerance:
            break
        F = ComputeResiduo(x)
        r = np.dot(dx.transpose(), dx)
        ## s^t . s
        t = np.dot(F, dx.transpose())
        B = B + t * (1.0 / r)
        x0 = x
        F0 = F


x = broyden(x0, 100, 1.0e-12)

0      0.61920292 0.88079708 3.70604662E-01
1      0.47419484 0.92098311 1.45259902E-01
2      0.36036933 0.94981664 1.15584887E-01
3      0.32682355 0.94872605 3.34483927E-02
4      0.32004103 0.94762896 6.86921604E-03
5      0.31963695 0.94754343 4.13027290E-04
6      0.31963158 0.94754193 5.57625518E-06
7      0.31963154 0.94754192 4.37597659E-08
8      0.31963154 0.94754191 7.56526367E-10
9      0.31963154 0.94754191 1.34199830E-12
10     0.31963154 0.94754191 0.00000000E+00


# Estratégia de Convergência Global
## Busca Linear (Line Search)

O passo de Newton $\mathbf{\Delta x}$ calculado para resolver um sistema de equações não lineares pode nos levar a uma aproximação $\mathbf{x}^{(k+1)}=\mathbf{x}^k+\mathbf{\Delta x}$ indesejável onde a norma do resíduo $\| \mathbf{F}(\mathbf{x}^k+\mathbf{\Delta x})\|$ pode exceder a norma de $\| \mathbf{F}(\mathbf{x}^k\|$ de modo que nenhum progresso será feito na busca da solução no sistema não linear.

O emprego de técnicas de globalização pode ser aplicada de forma a reduzir a norma de função residuo em cada iteração. Dessa forma, expandindo o domínio de convergência da iteração de Newton.

O método de Busca linear (**Line search**) substitui a atualização original do método de Newton por:

$$\mathbf{x}^{(k+1)}=\mathbf{x}^{(k)}+\lambda^{(k)}\mathbf{\Delta x} \quad \quad (1)$$

onde $\lambda^{(k)}$ é um numero real maior que zero. Em nossoas algoritmos até aqui aceitavamos $\mathbf{\Delta x}$ como a direção de busca para a solução do sistema não linear, mas na verdade, o que iremos fazer agora é usar um passo de tamanho reduzido $\lambda^{(k)}<1$.

**A questão é como escolher $\lambda^{(k)}$?**

Observe que para um sistema de equações lineares $\mathbf{F(x)=0}$ não existe como determinar antecipadamente se um dado
vetor será melhor ou pior para um aproximação que leva a solução exata. No entato, podemos introduzir uma função objetivo que nos de esse indicativo, por exemplo:

$$
\phi(\mathbf{x})=\frac{1}{2}\|\mathbf{F}(\mathbf{x})\|_2^2 \quad \quad (2)
$$
Para determinar $\lambda^{(k)}$, basta resolver o problema de minimização

$$
\min_{λ>0}ϕ(\mathbf{x}+λ \mathbf{\Delta x}) \quad \quad (3)
$$

ou seja, pode-se tornar a função objetivo tão pequena quanto possível ao longo da direção de busca. No entanto, resolver (2) com precisão raramente é justificado uma vez que $\lambda^{(k)}$ determina apenas a próxima iteração de Newton e não a resposta final.

Assim, um algoritmo prático de busca linear testa valores candidatos de $\lambda^k$ para verificar se $\phi(\mathbf{x}^k+\lambda \mathbf{s})$ é reduzido. Exigindo diminuição suficiente da função objetivo, temos:

$$
\phi(\mathbf{x}^k+\lambda \mathbf{\Delta x}) \le
\phi(\mathbf{x}^k) + \alpha\lambda\mathbf{\Delta x}^T\nabla\phi(\mathbf{x}^k) \quad \quad (3)
$$

com parametro $α∈(0,1)$ é uma forma comum de garantir o progresso. O valor padrão $α=10^{−4}$ corresponde a um critério de redução de norma fácil de satisfazer, mas essa busca pode falhar se (3) não for satisfeito após um certo número de reduções de $λ$ (pro exemplo, após 10 iterações).

A quantidade $\mathbf{\Delta x}^T\nabla\phi(\mathbf{x}^k)$ em (3) é a derivada direcional de $\phi(\mathbf{x}^k)$ na direção de busca $\mathbf{\Delta x}$. Para a função objetivo (2) isso é sempre negativo, se a equação  for resolvida com precisão:

$$
\mathbf{\Delta x}^T\nabla\phi(\mathbf{x}^k)=\mathbf{\Delta x}^T\mathbf{J}(\mathbf{x}^k)^T\mathbf{F}(\mathbf{x}^k)=(\mathbf{J}(\mathbf{x}^k)\mathbf{\Delta x})^T\mathbf{F}(\mathbf{x}^k)=−\|\mathbf{F}(\mathbf{x}^k)\|_2^2 \quad \quad (4)
$$


Observe que inequaldade (3) pode ser testada sem a necessidade de avaliar o Jacobiano.

O código abaixo implementa o algoritmos algoritmo de line search descrito no livro de Dennis and Schnabe, Numerical Methods for Nonlinear Equationsand Unconstrained Optimization", Prentice-Hall (1983), seção 6.3.2.



In [ ]:
import numpy as np
from numpy.linalg import norm


def linesearch(X, S):
    F = ComputeResiduo(X)
    fnorm = norm(F)
    XP = np.zeros([2, 1])
    f = fnorm**2

    slope = -fnorm * fnorm

    if slope > 0:
        initslope = -slope
    if slope == 0:
        slope = -1.0

    lambda_min = 1.0e-08
    lambda_ = 1.0
    alpha_ = 1.0e-4
    k = 0
    kmax = 6
    lambda_prev = 0
    gprev = 0
    while True:
        # backtrack step
        k = k + 1

        # compute x + \lamdda*s
        XP = X + lambda_ * S

        # Compute objective function
        F = ComputeResiduo(XP)
        gnorm = norm(F)
        g = gnorm**2

        if g <= (f + 2.0 * lambda_ * alpha_ * slope):
            break
        if k == kmax:
            break
        if lambda_ < lambda_min:
            break
        if k == 1:
            # first backtracking, quadratic
            lambda_temp = -slope / (g - f - 2.0 * slope)
        else:
            # all subsequent backtrackings, cubic fit
            t1 = 0.5 * (g - f) - lambda_ * slope
            t2 = 0.5 * (gprev - f) - lambda_prev * slope
            t3 = 1.0 / (lambda_ - lambda_prev)
            t4 = lambda_ * lambda_
            t5 = lambda_prev * lambda_prev
            a = t3 * (t1 / t4 - t2 / t5)
            b = t3 * (lambda_ * t2 / t5 - lambda_prev * t1 / t4)
            d = b * b - 3.0 * a * slope
            if d < 0.0:
                d = 0.0
            if a == 0.0:
                lambda_temp = -slope / (2.0 * b)
            else:
                lambda_temp = (-b + np.sqrt(d)) / (3.0 * a)

        lambda_prev = lambda_
        gprev = g

        print(" lambda_temp %14.12e" % (lambda_temp))
        if lambda_temp > 0.5 * lambda_:
            lambda_ = 0.5 * lambda_
        if lambda_temp <= 0.1 * lambda_:
            lambda_ = 0.1 * lambda_
        else:
            lambda_ = lambda_temp

    print("  Line search fnorm %12.12e - gnorm %14.12e" % (fnorm, gnorm))
    return lambda_


def NewtonLS(x0, kmax, tolerance):
    # print("%-5s  %-10.8s %-10.8s %-8.8s" % ("Iter", "x1", "x2", "error") )
    for i in range(kmax):
        J = ComputeExJacobian(x0)
        F = ComputeResiduo(x0)
        dx = np.linalg.solve(J, -F)
        lambda_ = linesearch(x0, dx)
        x = x0 + lambda_ * dx
        error = np.linalg.norm(x - x0) / np.linalg.norm(x)
        # print("Newton Step: %d  -- Relative Error: %12.12e" %(i,error))
        print("%-5d  %-8.8f %-8.8f %-8.8E  %8.8E" % (i, x[0], x[1], error, lambda_))
        if error < tolerance:
            break
        x0 = x
    return x


x0 = np.zeros([2, 1])
x0[0] = 10
x0[1] = 10
x = NewtonLS(x0, 50, 1.0e-8)

  Line search fnorm 2.425825877050e+08 - gnorm 8.924115013397e+07
0      9.50000000 0.55000000 9.94463012E-01  1.00000000E+00
  Line search fnorm 8.924115013397e+07 - gnorm 3.283003072525e+07
1      8.99999960 -72.22499318 9.99905611E-01  1.00000000E+00
  Line search fnorm 3.283003072525e+07 - gnorm 1.207748922069e+07
2      8.49999905 -35.62097710 9.99627103E-01  1.00000000E+00
  Line search fnorm 1.207748922069e+07 - gnorm 4.443057569049e+06
3      7.99999835 -16.92968771 9.98572913E-01  1.00000000E+00
  Line search fnorm 4.443057569049e+06 - gnorm 1.634507629645e+06
4      7.49999758 -6.84047949 9.95135785E-01  1.00000000E+00
  Line search fnorm 1.634507629645e+06 - gnorm 6.012991934020e+05
5      6.99999760 0.07001021 9.89744343E-01  1.00000000E+00
  Line search fnorm 6.012991934020e+05 - gnorm 2.374159604496e+05
6      6.49975417 -292.75485059 9.99994169E-01  1.00000000E+00
  Line search fnorm 2.374159604496e+05 - gnorm 8.420613608367e+04
7      5.99942328 -146.31808768 9.99976684

# Referências

1. J. E. Dennis, Jr. and Robert B. Schnabel, Numerical Methods for Unconstrained Optimization and Nonlinear Equations, 1996. Book Series Name:Classics in Applied Mathematics. SIAM.
2. C. T. Kelley.Iterative Methods for Linear and Nonlinear Equations, 1995, Book Series Name:Frontiers in Applied Mathematics. SIAM.
3. Jorge Nocedal, Stephen J. Wright, Numerical Optimization, 2006, Springer New York, NY.